# Almacenamiento de datos

## Introducción

Vamos a generar una BBDD Relacional para el almacenamiento de los registros que hemos obtenido hasta ahora.

En este momento del proyecto (mayo/23) prepararemos una base de datos con 2 Tablas y posteriormente generaremos una tercera tabla con las variables que habremos generado en el notebook de Engineering para generar #tags. 

## Librerías

In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import sqlite3
# import mysql.connector


In [2]:
os.chdir(os.path.split(os.getcwd())[0])
folder=os.getcwd()
folder

'd:\\Data_science\\Javier\\Repositorios\\Proyecto_tienda_online'

## Estructuras de la BBDD

![Estructura_tablas](../Resources/image/Relacion_tablas.jpg)

## 1. Construyendo la BBDD

### a.- Creando la estructura

In [3]:
# Conectamos con la base de datos
conn = sqlite3.connect("Data/online_shop.db")

# Obtenemos un cursor que utilizaremos para hacer las queries
cursor = conn.cursor()

### Funciones para obtener datos y ver en un dataframe

In [4]:
# Con esta función leemos los datos y lo pasamos a un DataFrame de Pandas
def sql_query(query):

    # Ejecuta la query
    cursor.execute(query)

    # Almacena los datos de la query 
    ans = cursor.fetchall()

    # Obtenemos los nombres de las columnas de la tabla
    names = [description[0] for description in cursor.description]

    return pd.DataFrame(ans,columns=names)

In [9]:
res = cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
for name in res:
    print(name[0])

PRODUCT
COMMENT
TAGS


#### PRODUCT

In [6]:
query = '''
CREATE TABLE PRODUCT  (
    ID VARCHAR (3),
    PRODUCT VARCHAR (100),
    SLOGAN VARCHAR (100),
    DESCRIPTION VARCHAR (250),
    CHARACTERISTICS VARCHAR (250),
    LISTA_URL VARCHAR (50),
    REGULAR_PRICE INT (3),
    DISCOUNT_PRICE INT (3),
    PRIMARY KEY (ID)
)

'''


cursor.execute(query)

#### COMMENTS

In [7]:
query = '''
CREATE TABLE COMMENT  (
    ID_COMMENT VARCHAR (5),
    ID_PRODUCT VARCHAR (3),
    DATE DATETIME (100),
    RATIO INT (2),
    USERS VARCHAR (50),
    COMMENT VARCHAR (250), 
    PRIMARY KEY (ID_COMMENT),
    FOREIGN KEY (ID_PRODUCT) REFERENCES PRODUCT (ID)
)

'''

cursor.execute(query)

#### TAGS

In [8]:
query = '''
CREATE TABLE TAGS  (
    ID VARCHAR (3),
    JUGUETES INT(1),
    MUEBLES INT(1),
    LENCERIA INT(1),
    MASCULINO INT(1),
    FEMININO INT(1),
    ANAL INT(1),
    BDSM INT(1),
    AMENITIES INT(1),
    PRIMARY KEY (ID),
    FOREIGN KEY (ID) REFERENCES PRODUCT (ID)
)

'''

# En esta tabla hay que convertir primeramente los booleanos en 0 y 1 para guardarla.
# También es importante tener esto en cuenta a la hora de extraer la información.
cursor.execute(query)

### b.- Cargando los datos dentro de la BBDD

In [10]:
df_products=pd.read_csv(r'Data\productos.csv')
print("Long. dataframe",len(df_products.columns))
df_products.head()

Long. dataframe 8


,ID,PRODUCT,SLOGAN,DESCRIPTION,CHARACTERISTICS,LISTA_URL,REGULAR_PRICE,DISCOUNT_PRICE
0,0,Desliz! Lubricante íntimo de agua 100ml,NaN,"Algunos lubricantes son un poco densos, otros ...",Bote de 100ml de venta exclusiva en amantis.ne...,https://www.amantis.net/desliz-lubricante-inti...,9.99,7.99
1,1,LIZO 2,Dildo de suave silicona en 3 tamaños,"En un azulejo de la cocina, en una puerta, en ...",Material: silicona médica de máxima calidadDim...,https://www.amantis.net/lizo-2-dildo-suave-sil...,59.99,17.99
2,2,FOXTAIL,plug anal cola de zorro de 35cm,Deja volar la imaginación y saca tu lado más s...,Tamaño plug S: 6.6cm Max Dia.: 2.8cmCola de pe...,https://www.amantis.net/foxtail-plug-anal-cola...,29.99,9.99
3,3,Vibrador Líquido con sabor Desliz! VIBRAGEL 30ml,hormigueo oral,¿Alguna vez has sentido un hormigueo en lo más...,"Ingredientes: agua, glicerina, hidroxietilcelu...",https://www.amantis.net/desliz-vibragel-liquid...,29.99,9.99
4,4,Bacanal FORTE TARRO 200ml,lubricante anal concentrado con aloe,Si alguna vez has pensado que el sexo anal es ...,Su textura no es grasa y no mancha. Compatible...,https://www.amantis.net/bacanal-forte-tarro-20...,24.99,24.99


In [11]:
df_comments=pd.read_csv(r'Data\comentarios.csv',index_col=False)
print("Long. dataframe",len(df_comments.columns))
df_comments.rename(columns={"ID":"ID_PRODUCT","Unnamed: 0":"ID_COMMENT"},inplace=True)
df_comments.head

Long. dataframe 6


<bound method NDFrame.head of        ID_COMMENT  ID_PRODUCT                        DATE  RATIO  \
0               0           0        jueves 18 mayo, 2023      5   
1               1           0    miércoles 12 abril, 2023      5   
2               2           0        lunes 10 abril, 2023      5   
3               3           0        lunes 23 enero, 2023      5   
4               4           0    miércoles 11 enero, 2023      5   
...           ...         ...                         ...    ...   
10328       11370         575      viernes 22 enero, 2016      5   
10329       11371         575  viernes 26 diciembre, 2014      4   
10330       11372         575   martes 09 diciembre, 2014      4   
10331       11373         575   martes 09 diciembre, 2014      5   
10332       11374         575   jueves 04 diciembre, 2014      5   

              USERS                                            COMMENT  
0             David  Muy recomendable. El bote cunde, hidrata muy b...  
1      

In [12]:
lista_products= df_products.values.tolist()                                                #   Pasamos a lista el dataframe

cursor.executemany("INSERT INTO PRODUCT VALUES (?,?,?,?,?,?,?,?)", lista_products)           #   Salvamos directamente todos los datos en la tabla PRODUCTS

lista_comments= df_comments.values.tolist()                                                #   Pasamos a lista el dataframe

cursor.executemany("INSERT INTO COMMENT VALUES (?,?,?,?,?,?)", lista_comments)                 #   Salvamos directamente todos los datos en la tabla COMMENTS

In [13]:
conn.commit()
cursor.close()
conn.close()

## 2. Realizando consultas para visualizar datos.

In [ ]:
# Conectamos con la base de datos
conn = sqlite3.connect("Data/online_shop.db")

# Obtenemos un cursor que utilizaremos para hacer las queries
cursor = conn.cursor()

In [ ]:
# # Establecer los parámetros de conexión
# config = {
#     'user': user,
#     'password': password,
#     'host': host,
#     'database_name':database_name,
#     'raise_on_warnings': True
# }

# # Establecer la conexión
# conn = mysql.connector.connect(**config)

# # Crear el cursor
# cursor = conn.cursor()

## Cerramos la BBDD.

In [47]:
conn.commit()
cursor.close()
conn.close()